In [207]:
import requests
import pandas as pd
from splinter import Browser
from time import sleep
from bs4 import BeautifulSoup
from math import ceil

In [4]:
bb_df = pd.read_csv('mlb_draft_player_salaries/mlb_draft_player_salaries_2009_2018.csv')

In [5]:
bb_df.head()

,Year,RD,Pick,Team,Pos,Player,School,Type,ST,Signed,Bonus
0,2009.0,1,1.0,Nationals,RHP,Stephen Strasburg,San Diego State,4YR,CA,Y,"$7,500,000"
1,2009.0,1,2.0,Mariners,OF,Dustin Ackley,North Carolina,4YR,NC,Y,"$6,000,000"
2,2009.0,1,3.0,Padres,OF,Donavan Tate,Cartersville (Ga.) HS,HS,GA,Y,"$6,250,000"
3,2009.0,1,4.0,Pirates,C,Tony Sanchez,Boston College,4YR,MA,Y,"$2,500,000"
4,2009.0,1,5.0,Orioles,RHP,Matt Hobgood,Norco (Calif.) HS,HS,CA,Y,"$2,422,000"


In [61]:
player_stats = soup.find_all('div', class_ = 'stats_pullout')

In [106]:
#takes a list_iterator object and pull the children from it returning a list
def children_to_list(list):
    result = []
    for l in list:
        result.append(l)
        
    return result # RETURNS LIST

In [102]:
get_children(player_stats[0].children)[1].find_all('p')

[<p><strong>2019</strong></p>, <p><strong>Career</strong></p>]

In [119]:
child_list = children_to_list(soup.find_all('div', class_ = 'stats_pullout')[0].children)

In [120]:
try:
    for i in range(len(child_list)):
        if child_list[i] == '\n':
            child_list.pop(i)
except:
    print('done')

done


In [133]:
stats = []

if len(child_list[0].find_all('p')) == 2:
    for child in child_list:
        p_list = child.find_all('p')
        for i in range(len(p_list)):
            if (i % 2 == 1):
                stats.append(p_list[i].text)
elif len(child_list[0].find_all('p')) == 1:
    for child in child_list:
        p_list = child.find_all('p')
        for p in p_list:
            stats.append[p.text]
else:
    print('no stats')

In [131]:
stats

['Career',
 '30.3',
 '101',
 '55',
 '3.16',
 '220',
 '220',
 '0',
 '1320.2',
 '1554',
 '1.088']

In [134]:
browser = Browser('chrome')    

In [181]:
test = player_list[1].split(' ')
test

['Dustin', 'Ackley']

In [182]:
test[1][0:5] + test[0][0:2]

'AckleDu'

In [183]:
#https://www.baseball-reference.com/players/s/solissa00.shtml
name = test[1][0:5] + test[0][0:2]
number = 1
url = f'https://www.baseball-reference.com/players/{name.lower()[0]}/{name.lower()}0{number}.shtml'
url

'https://www.baseball-reference.com/players/a/ackledu01.shtml'

In [167]:
r = requests.get('https://www.baseball-reference.com/players/a/ackledu01.shtml')

In [168]:
soup =  BeautifulSoup(r.text, 'html.parser')
soup.find_all('h1')

[<h1 itemprop="name">Dustin Ackley</h1>]

In [175]:
#RETURNS LIST
def scrape_player_data(player):
    name = test[1][0:5] + test[0][0:2]
    number = 0
    url = f'https://www.baseball-reference.com/players/{name.lower()[0]}/{name.lower()}0{number}.shtml'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    if '404' in soup.find_all('h1'):
        number += 1
        url = f'https://www.baseball-reference.com/players/{name.lower()[0]}/{name.lower()}0{number}.shtml'
        r = requests.get(url)
    
    stats = []   
    
    child_list = children_to_list(soup.find_all('div', class_ = 'stats_pullout')[0].children)
    
    if len(child_list[0].find_all('p')) == 2:
        for child in child_list:
            p_list = child.find_all('p')
            for i in range(len(p_list)):
                if (i % 2 == 1):
                    stats.append(p_list[i].text)
    elif len(child_list[0].find_all('p')) == 1:
        for child in child_list:
            p_list = child.find_all('p')
            for p in p_list:
                stats.append[p.text]
    return stats

In [170]:
df = pd.DataFrame(columns = ['C/S','war', 'ab', 'h', 'hr', 'ba', 'r', 'rbi', 'sb', 'obp', 'slg', 'ops', 'ops+'], index = [i for i in range(len(player_list))])

In [176]:
scrape_player_data(player_list[0])

['Career',
 '30.3',
 '101',
 '55',
 '3.16',
 '220',
 '220',
 '0',
 '1320.2',
 '1554',
 '1.088']

In [184]:
child_list = children_to_list(soup.find_all('div', class_ = 'stats_pullout')[0].children)

In [186]:
try:
    for i in range(len(child_list)):
        if child_list[i] == '\n':
            child_list.pop(i)
except:
    print('done')

done


In [208]:
x = 3/2
ceil(x)

2

In [197]:
len(child_list[1].find_all('p'))

5

In [210]:
stats = {}

if len(child_list[0].find_all('p')) == 2:
    
        for child in child_list:
            
            p_list = child.find_all('p')
            h_list = child.find_all('h4')
                
            for i in range(len(p_list)):
                if (i % 2 == 1):
                    stats[h_list[ceil(i/2)].text] = p_list[i].text
                    
elif len(child_list[0].find_all('p')) == 1:
    for child in child_list:
        p_list = child.find_all('p')
        h_list = child.find_all('h4')
        for i in range(len(p_list)):
            stats[h_list[i]] = p_list[i].text

In [214]:
stats.keys

<function dict.keys>